In [16]:
import skmob
import pandas as pd
import csv, os
from skmob.preprocessing import detection, clustering

# def get_week(device_id):
#     stdf = skmob.TrajDataFrame.from_file('./staypoint/53/stay_points.csv', 
#         latitude='latitude', longitude='longitude', user_id='deviceid', datetime='arrive_time')

#     stdf.rename(columns={'leaving_time': 'leaving_datetime'}, inplace=True)
#     stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])

#     weeks = [g for n, g in stdf.set_index(
#         'datetime').groupby(pd.Grouper(freq='W'))]

#     return weeks

device_id_list = []
for it in os.scandir('./staypoint'):
    if it.is_dir():
        if it.name.isdigit():
            device_id_list.append(it.name)
device_id_list.sort()

for device_id in device_id_list:
    stdf = skmob.TrajDataFrame.from_file('./staypoint/'+ device_id +'/stay_points.csv',
        latitude='latitude', longitude='longitude', user_id='deviceid', datetime='datetime')

    stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])

    # m = stdf.plot_trajectory(max_users=1, start_end_markers=False)
    # stdf.plot_stops(max_users=1, map_f=m)

    cstdf = clustering.cluster(stdf, cluster_radius_km=0.2, min_samples=1)

    stay_point_list = cstdf.values.tolist()
    for stay_point in stay_point_list:
        stay_point[3] = stay_point[3].strftime('%Y-%m-%d %H:%M:%S')
        stay_point[4] = stay_point[4].strftime('%Y-%m-%d %H:%M:%S')
        fields = ['deviceid', 'latitude', 'longitude','datetime', 'leaving_datetime', 'cluster']
        path = "./cluster/"+device_id
        if not os.path.isdir(path):
            os.mkdir(path)
        with open("./cluster/"+device_id+'/clustering_result.csv', 'w', newline='') as f:
            write = csv.writer(f)
            write.writerow(fields)
            write.writerows(stay_point_list)



# print(cstdf.values.tolist())

# f = open('test.txt', 'w')
# stdf.rename(columns={'leaving_time': 'leaving_datetime'}, inplace=True)
# stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])

# print(stdf.head())

# map_f = stdf.plot_trajectory(max_points=1000, start_end_markers=False)
# stdf.plot_stops(map_f=map_f)
